In [1]:
from PIL import Image
Image.CUBIC = Image.BICUBIC
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
import tkinter as tk
import os, time, sys, queue, datetime
from csv import writer
from datetime import datetime, timedelta
from threading import Thread, Event
from tkinter import filedialog, Canvas, Label, LabelFrame, Frame, PhotoImage, Button, Entry, Scrollbar, StringVar, IntVar, DoubleVar#, ttk 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import multiprocessing as mp
import http.server
import socketserver
from contextlib import closing
from more_itertools import time_limited

print("Done")

Done


In [2]:
# root window (parent to all)
class Windows(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.shared_data = {
            "geometry":    StringVar(self,"1280x800"),
            "padding":     IntVar(self, 10),
        }
        self.wm_title("Creative Envy Dashboard")
        ttk.Style("darkly")
        self.geometry(self.shared_data["geometry"].get())
        self.resizable(False, False)
        self.container = Frame(self)
        self.container.grid_rowconfigure(0, weight=1)
        self.container.grid_columnconfigure(0, weight=1)
        main_notebook = Notebook(self) # "self" as passed argument means Windows is parent 
        def on_tab_change(event):
            index = main_notebook.index(main_notebook.select())
        main_notebook.bind("<<NotebookTabChanged>>", on_tab_change)

class Notebook(ttk.Notebook): # child to main window
    # Tabs appear to add 30 pix to the resolution height
    def __init__(self, parent):
        ttk.Notebook.__init__(self, parent)
        for i, F in enumerate([Pane1, Pane2, Pane3]):
            frame = F(self, parent)
            self.add(frame, text= 'Tab %s' % (i + 1)) # "self." adds to this instance, ...(self) adds to parent instance
        self.grid(row=0, column=0, sticky="nsew")

class Pane1(tk.Frame): # child to Notebook / controller to widgets
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        # Pane1 Objects
        canvas1 = tk.Canvas(
            self, # needed to commit this widget to this Pane
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
        ).grid(row=1, column=0)
        canvas2 = tk.Canvas(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
        ).grid(row=1, column=1)
        canvas3 = tk.Canvas(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
        ).grid(row=1, column=2)
        canvas4 = tk.Canvas(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
        ).grid(row=1, column=3)
        canvas5 = tk.Canvas(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5)),
        ).grid(row=1, column=4)
        
        # AWS Temp
        meter1 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            # metersize=180,
            # padding=5,
            amountused=25,
            metertype="semi",
            subtext="Humidity",
            interactive=False,
        ).grid(row=2, column=0)
        
        # AWS Solar
        meter2 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=65,
            metertype="semi",
            subtext="UV Idx",
            interactive=False,
        ).grid(row=2, column=1)
        
        # AWS Wind Spd/Dir
        meter3 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=25,
            metertype="semi",
            subtext="Humidity",
            interactive=False,
        ).grid(row=2, column=2)
        
        # AWS Chicken Temp
        meter4 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=65,
            metertype="semi",
            subtext="UV Idx",
            interactive=False,
        ).grid(row=2, column=3)
        
        # Bee Wt.
        meter5 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=25,
            metertype="semi",
            subtext="Humidity",
            interactive=False,
        ).grid(row=2, column=4)
        
        # AWS Humid
        meter6 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=65,
            metertype="semi",
            subtext="UV Idx",
            interactive=False,
        ).grid(row=3, column=0)
        
        # AWS UV Index
        meter7 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=25,
            metertype="semi",
            subtext="Humidity",
            interactive=False,
        ).grid(row=3, column=1)
        
        
        # AWS Precip
        meter8 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=65,
            metertype="semi",
            subtext="UV Idx",
            interactive=False,
        ).grid(row=3, column=2)
        
        # Bee Temp
        meter9 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=25,
            metertype="semi",
            subtext="Humidity",
            interactive=False,
        ).grid(row=3, column=3)
        
        # Bee Humid
        meter10 = ttk.Meter(
            self,
            metersize=int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 10)) / 5),
            padding=self.controller.shared_data["padding"].get(),
            amountused=65,
            metertype="semi",
            subtext="UV Idx",
            interactive=False,
        ).grid(row=3, column=4)

class Pane2(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        # Pane2 Objects
        canvas1 = tk.Canvas(
            self,
            width = str(int(self.controller.shared_data["geometry"].get().split("x")[0])),
            height = str(int(self.controller.shared_data["geometry"].get().split("x")[1])),
        ).grid(row=1, column=0)

class Pane3(tk.Frame): # Picture Frame (need to install pyicloud)
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        # Pane3 Objects
        canvas1 = tk.Canvas(
            self,
            width = str(int(self.controller.shared_data["geometry"].get().split("x")[0])),
            height = str(int(self.controller.shared_data["geometry"].get().split("x")[1])),
        )
        canvas1.create_rectangle(0, 0, int(self.controller.shared_data["geometry"].get().split("x")[0]), int(self.controller.shared_data["geometry"].get().split("x")[1]), fill="#444444")
        canvas1.create_rectangle(20, 20, int(self.controller.shared_data["geometry"].get().split("x")[0]) - 20, int(self.controller.shared_data["geometry"].get().split("x")[1]) - 50, fill="black")
        canvas1.grid(row=1, column=0)
    
def main(): # creates main window and calls main loop
    windows = Windows()
    windows.mainloop()
    
if __name__ == '__main__': # runs main if in python script
    main()